In [2]:
import os
import cv2
import numpy as np
from albumentations import (
    Compose, GaussianBlur, MotionBlur, MedianBlur, GaussNoise, RandomBrightnessContrast
)

# Define augmentation pipeline
def get_augmentation_pipeline():
    return Compose([
        GaussianBlur(blur_limit=(3, 7), p=0.5),
        MotionBlur(blur_limit=7, p=0.5),
        MedianBlur(blur_limit=7, p=0.5),
        GaussNoise(var_limit=(10.0, 50.0), p=0.5),
        RandomBrightnessContrast(p=0.2)
    ])

# Paths
not_blur_dir = 'not_blur'
blurred_dir = 'blurred'
os.makedirs(blurred_dir, exist_ok=True)

# Augment and save blurred images
augmentation_pipeline = get_augmentation_pipeline()

for img_name in os.listdir(not_blur_dir):
    img_path = os.path.join(not_blur_dir, img_name)
    image = cv2.imread(img_path)

    augmented = augmentation_pipeline(image=image)
    blurred_image = augmented['image']

    blurred_img_path = os.path.join(blurred_dir, img_name)
    cv2.imwrite(blurred_img_path, blurred_image)


In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input
from tensorflow.keras.models import Model

def create_deblurring_model():
    input_img = Input(shape=(None, None, 3))

    # Encoder
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)

    # Decoder
    x = Conv2DTranspose(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(x)

    model = Model(input_img, x)
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

model = create_deblurring_model()
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 64)    1792      
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 64)    36928     
                                                                 
 conv2d_transpose (Conv2DTr  (None, None, None, 64)    36928     
 anspose)                                                        
                                                                 
 conv2d_transpose_1 (Conv2D  (None, None, None, 3)     1731      
 Transpose)                                                      
                                                                 
Total params: 77379 (302.26 KB)
Trainable params: 77379 (302.

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data generators
datagen = ImageDataGenerator()

not_blur_generator = datagen.flow_from_directory(
    'not_blur',
    target_size=(256, 256),
    batch_size=16,
    class_mode=None,
    color_mode='rgb',
    shuffle=True,
    seed=42
)

blurred_generator = datagen.flow_from_directory(
    'blurred',
    target_size=(256, 256),
    batch_size=16,
    class_mode=None,
    color_mode='rgb',
    shuffle=True,
    seed=42
)

def generator(not_blur_gen, blur_gen):
    while True:
        not_blur_img = not_blur_gen.next()
        blur_img = blur_gen.next()
        yield (blur_img, not_blur_img)

train_generator = generator(not_blur_generator, blurred_generator)

# Train the model
model.fit(train_generator, steps_per_epoch=100, epochs=50)


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Epoch 1/50
100/100 [==============================] - 2s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 2/50
100/100 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 3/50
100/100 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 4/50
100/100 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 5/50
100/100 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 6/50
100/100 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 7/50
100/100 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 8/50
100/100 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 9/50
100/100 [==============================] - 0s 2

In [5]:
import matplotlib.pyplot as plt

# Load a blurred test image
test_img_path = 'blurred/0000000_orig.png'
test_img = cv2.imread(test_img_path)
test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
test_img = cv2.resize(test_img, (256, 256))
test_img = np.expand_dims(test_img, axis=0)

# Deblur the image using the trained model
deblurred_img = model.predict(test_img)
deblurred_img = np.squeeze(deblurred_img)

deblurred_img = cv2.cvtColor(deblurred_img, cv2.COLOR_RGB2BGR)

# Display the original blurred and deblurred images
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Blurred Image')
plt.imshow(test_img[0])

plt.subplot(1, 2, 2)
plt.title('Deblurred Image')
plt.imshow(deblurred_img)
plt.show()


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
